In [19]:
import os
import shutil

original_db = "database/hotel_reservation_sample.db"
working_db = "database/hotel_reservation.db"
shutil.copy(original_db, working_db)

os.environ["DB"] = working_db

# Userstory 3.1
Ich möchte neue Hotels zum System hinzufügen

## Input Daten vom Admin

* Daten wie Strasse, Hotelname, Anzahl Sterne und Ort(Addresse)




In [20]:
# Userstory 3.1
from datetime import date
from business_logic.hotel_manager import HotelManager
from business_logic.address_manager import AddressManager
from model.hotel import Hotel
from model.address import Address

address_manager = AddressManager()
createded_address = address_manager.create_address("Teststrasse", "Zürich", 8001)
print(f"Neue Addresse: {createded_address.street}, {createded_address.city}, {createded_address.zip_code}")


hotel_manager = HotelManager()
h = Hotel(6, "Olympia", 5)
hotel = hotel_manager.create_hotel("Olympia", 5, createded_address)
print(f"Neues Hotel: {h.name}, Sterne: {h.stars}")

Neue Addresse: Teststrasse, Zürich, 8001
Neues Hotel: Olympia, Sterne: 5


# User Story 4
Als Gast möchte ich ein Zimmer in einem bestimmten Hotel
buchen, um meinen Urlaub zu planen.


In [21]:
from business_logic.booking_manager import BookingManager
from model.guest import Guest
from model.room import Room
from model.room_type import RoomType
from datetime import date


guest = Guest(1, "John", "Doe", "mail")
room = Room(1, 202, 80.90)
room_type = RoomType(1, "Standard", 1)
room.room_type = room_type

booking_manager = BookingManager()
b = booking_manager.create_booking(guest, room,1, date(2025, 1, 1), date(2025, 1, 3))
print(f" Checkin Date: {b.check_in_date}, Amount: {b.total_amount}")

 Checkin Date: 2025-01-01, Amount: 161.8
